## Solution
The creation of the model will be carried out according to the following plan


1.   Features analysis
2.   Classification
3.   Quality Assessment 

## 1. Feature Analysis

For a better understanding of the problem, let's turn to the primary source - the article by Johnson et al. 

The study area for this article was a forested area in Ibaraki Prefecture, Japan, approximately 13 km × 12 km. The landscape consists mainly
of Cryptomeria japonica (Sugi, or Japanese Cedar) planted forest, Chamaecyparis obtusa (Hinoki, or Japanese Cypress) planted forest and mixed deciduous broadleaf natural forest, with a small amount of other land use/land cover types scattered throughout the image (agriculture, roads, buildings and so on).

Orthorectified ASTER imagery was acquired for three dates in different seasons (26 September 2010, 19 March 2011 and 8 May 2011) to provide adequate spectral **discrimination between coniferous and deciduous tree species**)

<img src="forest.png">